In [ ]:
import numpy as np
import nltk
from stop_words import get_stop_words
from nltk.corpus import stopwords
import re
from nltk.stem.porter import *
from collections import Counter
from collections import defaultdict
from matplotlib import pyplot as plt



In [ ]:
def preprocess(sent):
    stemmer = PorterStemmer()
    #tokenization
    words = str(sent).split()
    new_words = []
    for w in words:
        #lowercase
        w = w.lower()
        #remove stopwords, you can change the stopwords list in here
        if w not in stop_words:
            w = stemmer.stem(w)
            new_words.append(w)
        
    return ' '.join(new_words)

In [ ]:
with open('sample.txt') as f:
    
    